In [20]:
import numpy as np 
from numpy import random
import pandas as pd 
#import seaborn as sns
import scipy 
from scipy.stats import chi2
import math
import sys
from scipy.stats import beta
import matplotlib.pyplot as plt

### The Gaussian case

In [21]:

p = 5;
m= 10; 
N=   100
v=min(p,m)
b=1;c=1
K_st_st=1
a_Quad_loss = 1 / (K_st_st*( m + p + 1)) 
t_K_Quad_loss =  ( 2*(p+m-2*v-1)*( (p+m+1)*((c/b)**2) - p - m +v -2 ) )/((p+m - 2*v+1)*(p+m-2*v+3) ) 

Loss_Sigma_a_Quad = 0
Loss_Sigma_K_Quad = 0 


#------------Structure de Sigma-----------
"-------------Identity----------------------"
#Sigma=np.identity(p)

"------------Random--------------------"
#e =p*np.random.rand(p,p)
#Sigma= np.dot(e.T,e)
"-----------Block----------------------"
#gamma=1.9
#sigma=np.zeros(p)
#for i in range(0,p):
#    sigma[p-1-i]=gamma**i
#Sigma=np.diag(sigma)
"----------AR-----------------------"
Sigma= np.zeros((p,p))
rho=0.9

for ii in range(0,p):
    for jj in range(0,p):
        Sigma[ii,jj] = rho**abs(ii-jj)
"-----------------------------------------"

#w, V = np.linalg.eig(Sigma)
#Sigma=np.diag(w)
Sigma_inv=np.linalg.inv(Sigma)

for j in range(0,N): 
    U=np.random.multivariate_normal(np.zeros(p), Sigma,m)
    S = np.dot(U.T,U) 
    S_plus = np.linalg.pinv(S) 
    trace_S_plus = np.matrix.trace(S_plus) 
    mu = 1/ trace_S_plus
    [R,V,D] = np.linalg.svd(U)                   # SVD pour U=R'VD afin de construire S= H L_tilde H^T
    #val_propre_S = V*V.T
    L = np.diag(V*V )                          # Recuperer L
    H1 = D.T[:,0:v]
    Sigma_hat_a_Quad_loss  =   a_Quad_loss *S
    Phi_K_Quad = L + mu * t_K_Quad_loss * np.identity(v)
    Sigma_hat_K =  a_Quad_loss  * H1@Phi_K_Quad@H1.T # Estimateur de Konno
    Loss_Sigma_K_Quad= Loss_Sigma_K_Quad +np.matrix.trace( ( ( Sigma_inv@Sigma_hat_K  )-np.identity(p) ) @  (  Sigma_inv@Sigma_hat_K  -np.identity(p) ))
    Risk_Sigma_K_Quad=(1/N)*(Loss_Sigma_K_Quad)
    
    Loss_Sigma_a_Quad = Loss_Sigma_a_Quad + np.matrix.trace((   Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ) @  ( Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ))
    Risk_Sigma_a_Quad=(1/N)*(Loss_Sigma_a_Quad)
Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)
print('Cout Q:','Risk aS =',Risk_Sigma_a_Quad,"||" ,'risk Konno =', Risk_Sigma_K_Quad,"||",'Prial  = ',Prial_K_a_Quad)








Cout Q: Risk aS = 1.8837088988638144 || risk Konno = 1.759124800076145 || Prial  =  6.6137660050772205


#### The mixture of Beta case

In [3]:

p= 20;
m = 25 ;
N=   10
v=min(p,m)
alpha= p/2 + .1 #alpha > p\2
Beta=  .001    #beta > 0
b=1
c=(alpha- p/2 ) / (alpha - p/2 + Beta)
K_st_st=beta.moment(2, alpha, Beta, loc=0, scale=1)
a_Quad_loss = 1 / (K_st_st*( m + p + 1)) 
t_K_Quad_loss =  ( 2*(p+m-2*v-1)*( (p+m+1)*((c/b)**2) - p - m +v -2 ) )/((p+m - 2*v+1)*(p+m-2*v+3) ) 

Loss_Sigma_a_Quad = 0
Loss_Sigma_K_Quad = 0 


#------------Structure de Sigma-----------
"-------------Identity----------------------"
Sigma=np.identity(p)
"------------Random--------------------"
#e =p*np.random.rand(p,p)
#Sigma= np.dot(e.T,e)
"-----------Block----------------------"
#gamma=1.9
#sigma=np.zeros(p)
#for i in range(0,p):
#    sigma[p-1-i]=gamma**i
#Sigma=np.diag(sigma)
"----------AR-----------------------"
#Sigma= np.zeros((p,p))
#rho=0.9

#for ii in range(0,p):
#    for jj in range(0,p):
#        Sigma[ii,jj] = rho**abs(ii-jj)
"-----------------------------------------"

for j in range(0,N): 
   # U=np.random.multivariate_normal(np.zeros(p), Sigma,m)
    R=np.sqrt(np.random.beta(alpha, Beta, p));
    RR=np.tile(R,(m, 1))
    X=np.random.multivariate_normal(np.zeros(p), Sigma, m)
    U= np.multiply(X,RR)
    
    S = np.dot(U.T,U) 
    S_plus = np.linalg.pinv(S) 
    trace_S_plus = np.matrix.trace(S_plus) 
    mu = 1/ trace_S_plus
    [R,V,D] = np.linalg.svd(U)                   # SVD pour U=R'VD afin de construire S= H L_tilde H^T
    #val_propre_S = V*V.T
    L = np.diag(V*V )                          # Recuperer L
    H1 = D.T[:,0:v]
    Sigma_hat_a_Quad_loss  =   a_Quad_loss *S
    Phi_K_Quad = L + mu * t_K_Quad_loss * np.identity(v)
    Sigma_hat_K =  a_Quad_loss  * H1@Phi_K_Quad@H1.T # Estimateur de Konno
    Loss_Sigma_K_Quad= Loss_Sigma_K_Quad +np.matrix.trace( ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_K ) )-np.identity(p) ) @  ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_K ) )-np.identity(p) ))
    Risk_Sigma_K_Quad=(1/N)*(Loss_Sigma_K_Quad)
    Loss_Sigma_a_Quad = Loss_Sigma_a_Quad + np.matrix.trace( ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_a_Quad_loss) )-np.identity(p) ) @  ( ( (np.linalg.inv(Sigma))@ (Sigma_hat_a_Quad_loss ) )-np.identity(p) ))
    Risk_Sigma_a_Quad=(1/N)*(Loss_Sigma_a_Quad)
Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)
print('Cout Q:','Risk aS =',Risk_Sigma_a_Quad,"||" ,'risk Konno =', Risk_Sigma_K_Quad,"||",'Prial  = ',Prial_K_a_Quad)



Cout Q: Risk aS = 9.224889642278375 || risk Konno = 8.952971896843781 || Prial  =  2.947653099158763


#### Comparaison of Prials

In [29]:
np.random.seed(100)

m=20 ; p=25;
N=   1000
v=min(p,m)
b=1;c=1
K_st_st=1
a_Quad_loss = 1 / (K_st_st*( m + p + 1)) 
t_K_Quad_loss =  ( 2*(p+m-2*v-1)*( (p+m+1)*((c/b)**2) - p - m +v -2 ) )/((p+m - 2*v+1)*(p+m-2*v+3) ) 
Loss_Sigma_a_Quad = 0
Loss_Sigma_K_Quad = 0 
#------------Structure de Sigma-----------
"-------------Identity----------------------"
Sigma=np.identity(p)
"------------Random--------------------"
#e =p*np.random.rand(p,p)
#Sigma= np.dot(e.T,e)
"-----------Block----------------------"
#gamma=1.2
#sigma=np.zeros(p)
#for i in range(0,p):
#    sigma[p-1-i]=gamma**i
#Sigma=np.diag(sigma)
"----------AR-----------------------"
#Sigma= np.zeros((p,p))
#rho=0.9
#for ii in range(0,p):
#    for jj in range(0,p):
#        Sigma[ii,jj] = rho**abs(ii-jj)

"-------Diagnoal Tsukuma--------"

#Sigma= np.zeros((p,p))
#xx=np.zeros(p)
#for i in range(0,p):
#    xx[i]= 100**(1-(p-i)/p)
#    xx[0]=100
#Sigma=np.diag(-1*np.sort(-xx))

"-----------------------------------------"
Sigma_inv=np.linalg.inv(Sigma)

for j in range(0,N): 
    U=np.random.multivariate_normal(np.zeros(p), Sigma,m)
    S = np.dot(U.T,U) 
    S_plus = np.linalg.pinv(S) 
    trace_S_plus = np.matrix.trace(S_plus) 
    mu = 1/ trace_S_plus
    [R,V,D] = np.linalg.svd(U)                   # SVD pour U=R'VD afin de construire S= H L_tilde H^T
    #val_propre_S = V*V.T
    L = np.diag(V*V )                          # Recuperer L
    H1 = D.T[:,0:v]
    Sigma_hat_a_Quad_loss  =   a_Quad_loss *S
    Phi_K_Quad = L + mu * t_K_Quad_loss * np.identity(v)
    Sigma_hat_K =  a_Quad_loss  * H1@Phi_K_Quad@H1.T # Estimateur de Konno
    Loss_Sigma_K_Quad= Loss_Sigma_K_Quad +np.matrix.trace( ( ( Sigma_inv@Sigma_hat_K  )-np.identity(p) ) @  (  Sigma_inv@Sigma_hat_K  -np.identity(p) ))
    Risk_Sigma_K_Quad=(1/N)*(Loss_Sigma_K_Quad)
    Loss_Sigma_a_Quad = Loss_Sigma_a_Quad + np.matrix.trace((   Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ) @  ( Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ))
    Risk_Sigma_a_Quad=(1/N)*(Loss_Sigma_a_Quad)
Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)
Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)
#print('Cout Q:','Risk aS =',Risk_Sigma_a_Quad,"||" ,'risk Konno =', Risk_Sigma_K_Quad,"||",'Prial  = ',Prial_K_a_Quad)


print(round(Prial_K_a_Quad,4))


alpha=  p/2 + .1 #alpha > p\2
Beta=  .001    #beta > 0
b=1
c=(alpha- p/2 ) / (alpha - p/2 + Beta)
K_st_st=beta.moment(2, alpha, Beta, loc=0, scale=1)
a_Quad_loss = 1 / (K_st_st*( m + p + 1)) 
t_K_Quad_loss =  ( 2*(p+m-2*v-1)*( (p+m+1)*((c/b)**2) - p - m +v -2 ) )/((p+m - 2*v+1)*(p+m-2*v+3) ) 
Loss_Sigma_a_Quad = 0
Loss_Sigma_K_Quad = 0
for j in range(0,N): 
   # U=np.random.multivariate_normal(np.zeros(p), Sigma,m)
    R=np.sqrt(np.random.beta(alpha, Beta, p));
    RR=np.tile(R,(m, 1))
    X=np.random.multivariate_normal(np.zeros(p), Sigma, m)
    U= np.multiply(X,RR)
    
    S = np.dot(U.T,U) 
    S_plus = np.linalg.pinv(S) 
    trace_S_plus = np.matrix.trace(S_plus) 
    mu = 1/ trace_S_plus
    [R,V,D] = np.linalg.svd(U)                   # SVD pour U=R'VD afin de construire S= H L_tilde H^T
    #val_propre_S = V*V.T
    L = np.diag(V*V )                          # Recuperer L
    H1 = D.T[:,0:v]
    Sigma_hat_a_Quad_loss  =   a_Quad_loss *S
    Phi_K_Quad = L + mu * t_K_Quad_loss * np.identity(v)
    Sigma_hat_K =  a_Quad_loss  * H1@Phi_K_Quad@H1.T # Estimateur de Konno
    Loss_Sigma_K_Quad= Loss_Sigma_K_Quad +np.matrix.trace( ( ( Sigma_inv@Sigma_hat_K  )-np.identity(p) ) @  (  Sigma_inv@Sigma_hat_K  -np.identity(p) ))
    Risk_Sigma_K_Quad=(1/N)*(Loss_Sigma_K_Quad)
    Loss_Sigma_a_Quad = Loss_Sigma_a_Quad + np.matrix.trace((   Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ) @  ( Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ))
    Risk_Sigma_a_Quad=(1/N)*(Loss_Sigma_a_Quad)
Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)
#print('Cout Q:','Risk aS =',Risk_Sigma_a_Quad,"||" ,'risk Konno =', Risk_Sigma_K_Quad,"||",'Prial  = ',Prial_K_a_Quad)
print(round(Prial_K_a_Quad,4)) 

1.9455
1.8806


###tests estimateur de Tsukuma et kubokawa $()$

In [31]:

p = 5;
m= 10; 
N=   100
v=min(p,m)

b=1;c=1
K_st_st=1
a_Quad_loss = 1 / (K_st_st*( m + p + 1)) 
t_K_Quad_loss =  ( 2*(p+m-2*v-1)*( (p+m+1)*((c/b)**2) - p - m +v -2 ) )/((p+m - 2*v+1)*(p+m-2*v+3) ) 

Loss_Sigma_a_Quad = 0
Loss_Sigma_K_Quad = 0 


#------------Structure de Sigma-----------
"-------------Identity----------------------"
#Sigma=np.identity(p)

"------------Random--------------------"
#e =p*np.random.rand(p,p)
#Sigma= np.dot(e.T,e)
"-----------Block----------------------"
#gamma=1.9
#sigma=np.zeros(p)
#for i in range(0,p):
#    sigma[p-1-i]=gamma**i
#Sigma=np.diag(sigma)
"----------AR-----------------------"
#Sigma= np.zeros((p,p))
#rho=0.9

#for ii in range(0,p):
#    for jj in range(0,p):
#        Sigma[ii,jj] = rho**abs(ii-jj)
"-----------------------------------------"

#w, V = np.linalg.eig(Sigma)
#Sigma=np.diag(w)
Sigma_inv=np.linalg.inv(Sigma)

for j in range(0,N): 
    U=np.random.multivariate_normal(np.zeros(p), Sigma,m)
    S = np.dot(U.T,U) 
    S_plus = np.linalg.pinv(S) 
    trace_S_plus = np.matrix.trace(S_plus) 
    mu = 1/ trace_S_plus
    [R,V,D] = np.linalg.svd(U)                   # SVD pour U=R'VD afin de construire S= H L_tilde H^T
    #val_propre_S = V*V.T
    L = np.diag(V*V )                          # Recuperer L
    H1 = D.T[:,0:v]
    Sigma_hat_a_Quad_loss  =   a_Quad_loss *S
    Phi_K_Quad = L + mu * t_K_Quad_loss * np.identity(v)
    Sigma_hat_K =  a_Quad_loss  * H1@Phi_K_Quad@H1.T # Estimateur de Konno
    Loss_Sigma_K_Quad= Loss_Sigma_K_Quad +np.matrix.trace( ( ( Sigma_inv@Sigma_hat_K  )-np.identity(p) ) @  (  Sigma_inv@Sigma_hat_K  -np.identity(p) ))
    Risk_Sigma_K_Quad=(1/N)*(Loss_Sigma_K_Quad)
    
    Loss_Sigma_a_Quad = Loss_Sigma_a_Quad + np.matrix.trace((   Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ) @  ( Sigma_inv@Sigma_hat_a_Quad_loss -np.identity(p) ))
    Risk_Sigma_a_Quad=(1/N)*(Loss_Sigma_a_Quad)
Prial_K_a_Quad = 100*( (Risk_Sigma_a_Quad - Risk_Sigma_K_Quad   ) / Risk_Sigma_a_Quad)
print('Cout Q:','Risk aS =',Risk_Sigma_a_Quad,"||" ,'risk Konno =', Risk_Sigma_K_Quad,"||",'Prial  = ',Prial_K_a_Quad)









-366.61458873352444

In [41]:
c=4
((L[1,1]**(c+1) - L[2,2]**(c+1) ) / (L[1,1]- L[2,2]) )> (L[1,1]**c + L[2,2]**c )

True

In [52]:
c=-4
L=1./L
((L[1,1]**(c+1) - L[2,2]**(c+1) ) / (L[1,1]- L[2,2]) ) < (L[1,1]**c + L[2,2]**c )

/Users/macbookpro/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app


True

In [54]:
np.range([start, ]stop, [step]) 

array([[0.0123408 ,        inf,        inf,        inf,        inf],
       [       inf, 0.19235456,        inf,        inf,        inf],
       [       inf,        inf, 0.60888865,        inf,        inf],
       [       inf,        inf,        inf, 2.51821936,        inf],
       [       inf,        inf,        inf,        inf, 6.54708093]])